<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset/challenge). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more manageable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
*Successfully complete these all these objectives to earn a 2. There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

In [1]:
import pandas as pd

yelp = pd.read_json('./data/review_sample.json', lines=True)

In [2]:
yelp.head()

business_id  cool                date  funny  \
0  nDuEqIyRc8YKS1q1fX0CZg     1 2015-03-31 16:50:30      0   
1  eMYeEapscbKNqUDCx705hg     0 2015-12-16 05:31:03      0   
2  6Q7-wkCPc1KF75jZLOTcMw     1 2010-06-20 19:14:48      1   
3  k3zrItO4l9hwfLRwHBDc9w     3 2010-07-13 00:33:45      4   
4  6hpfRwGlOzbNv7k5eP9rsQ     1 2018-06-30 02:30:01      0   

                review_id  stars  \
0  eZs2tpEJtXPwawvHnHZIgQ      1   
1  DoQDWJsNbU0KL1O29l_Xug      4   
2  DDOdGU7zh56yQHmUnL1idQ      3   
3  LfTMUWnfGFMOfOIyJcwLVA      1   
4  zJSUdI7bJ8PNJAg4lnl_Gg      4   

                                                text  useful  \
0  BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...      10   
1  Came here for lunch Togo. Service was quick. S...       0   
2  I've been to Vegas dozens of times and had nev...       2   
3  We went here on a night where they closed off ...       5   
4  3.5 to 4 stars\n\nNot bad for the price, $12.9...       5   

                  user_id  
0  n1LM36qNg4rqGXIcvVXv8w  
1  5CgjjDAic2-FAvCtiHpytA  
2  BdV-cf3LScmb8kZ7iiBcMA  
3  cZZnBqh4gAEy4CdNvJailQ  
4  n9QO4ClYAS7h9fpQwa5bhA

## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [3]:
#Importing warnings to ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [4]:
#Import Statements

import numpy as np
import gensim
import os
import re

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

import pandas as pd

from nltk.corpus import stopwords
import nltk
import spacy
nlp= spacy.load("en_core_web_lg")

In [5]:
def tokenize(doc):
    
    list = []
    
    text = nlp(doc)
    
    for token in text:
        if ((token.is_punct == False) and (token.is_stop==True)) and (token.pos_!= 'PRON'):
            list.append(token.lemma_)
    return list

In [6]:
yelp['tokens']=yelp['text'].apply(tokenize)

In [7]:
yelp['tokens'].head()

0    [also, own, a, in, CA, and, a, for, with, an, ...
1    [here, for, be, be, no, here, be, can, be, a, ...
2    [have, be, to, of, and, have, never, into, for...
3    [here, on, a, where, off, part, of, the, to, h...
4    [to, not, for, the, for, get, off, at, the, fr...
Name: tokens, dtype: object

In [8]:
#analyze the tokens
from collections import Counter

word_counts = Counter()

yelp['tokens'].apply(lambda x: word_counts.update(x))

word_counts.most_common(20)

[('be', 61331),
 ('the', 55436),
 ('and', 38799),
 ('a', 28278),
 ('to', 27825),
 ('-PRON-', 22381),
 ('have', 17472),
 ('of', 16168),
 ('not', 14440),
 ('for', 13363),
 ('in', 12677),
 ('that', 10082),
 ('with', 9349),
 ('this', 9056),
 ('do', 8926),
 ('but', 8296),
 ('on', 7859),
 ('so', 6022),
 ('at', 5778),
 ('as', 4709)]

## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews
2. Write a fake review and query for the 10 most similar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, it will probably be best to use a `NearestNeighbors` model for this. 

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
data = yelp['text']

#Apply CountVectorizer to our Data
vect = CountVectorizer(stop_words = 'english', min_df = 0.05, max_df = 0.90)

#learn our Vocab
vect.fit(data)

#Get Sparse dtm
dtm = vect.transform(data)

In [12]:
df = pd.DataFrame(dtm.todense(),columns=vect.get_feature_names())
df.head()

10  amazing  area  asked  away  awesome  bad  bar  best  better  ...  \
0   0        0     0      0     0        0    0    0     0       0  ...   
1   0        0     0      0     0        0    0    0     0       0  ...   
2   0        0     1      0     0        0    0    0     0       0  ...   
3   0        0     0      0     0        0    0    0     1       0  ...   
4   0        0     0      0     0        0    1    2     0       0  ...   

   vegas  wait  want  wanted  wasn  way  went  work  worth  years  
0      0     1     0       0     0    0     0     0      0      0  
1      0     0     0       0     0    0     0     0      0      0  
2      2     0     0       0     0    0     1     0      0      0  
3      0     0     0       0     0    0     1     0      0      0  
4      0     0     0       0     0    0     0     0      0      0  

[5 rows x 116 columns]

In [15]:
#TFIDFVECTORIZER
from sklearn.feature_extraction.text import TfidfVectorizer

#Instantiate vectorizer object
tfidf=TfidfVectorizer(stop_words='english',max_features=5000)

#Create a vocab & get count/doc
new_dtm = tfidf.fit_transform(data)

#get feature names to use as df

dff = pd.DataFrame(new_dtm.todense(), columns=tfidf.get_feature_names())

#View Matrix
dff.head()

00  000  00pm   07   10  100  1000  101  10pm   11  ...  yuck  yuk  yum  \
0  0.0  0.0   0.0  0.0  0.0  0.0   0.0  0.0   0.0  0.0  ...   0.0  0.0  0.0   
1  0.0  0.0   0.0  0.0  0.0  0.0   0.0  0.0   0.0  0.0  ...   0.0  0.0  0.0   
2  0.0  0.0   0.0  0.0  0.0  0.0   0.0  0.0   0.0  0.0  ...   0.0  0.0  0.0   
3  0.0  0.0   0.0  0.0  0.0  0.0   0.0  0.0   0.0  0.0  ...   0.0  0.0  0.0   
4  0.0  0.0   0.0  0.0  0.0  0.0   0.0  0.0   0.0  0.0  ...   0.0  0.0  0.0   

   yummy  yup  zero  zone  zoo  zucchini  était  
0    0.0  0.0   0.0   0.0  0.0       0.0    0.0  
1    0.0  0.0   0.0   0.0  0.0       0.0    0.0  
2    0.0  0.0   0.0   0.0  0.0       0.0    0.0  
3    0.0  0.0   0.0   0.0  0.0       0.0    0.0  
4    0.0  0.0   0.0   0.0  0.0       0.0    0.0  

[5 rows x 5000 columns]

In [16]:
#NearestNeighbors
from sklearn.neighbors import NearestNeighbors

nn = NearestNeighbors(n_neighbors = 5, algorithm = 'ball_tree')

#fit NN
nn.fit(new_dtm)

NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [18]:
#Query using Kneighbors
nn.kneighbors(new_dtm)

(array([[0.        , 1.        , 1.        , 1.25325603, 1.25954602],
        [0.        , 1.        , 1.        , 1.17772553, 1.19154405],
        [0.        , 1.        , 1.        , 1.06839889, 1.11038143],
        ...,
        [0.        , 1.        , 1.        , 1.22137603, 1.2229785 ],
        [0.        , 1.        , 1.        , 1.24537508, 1.24737227],
        [0.        , 1.        , 1.        , 1.21437929, 1.22620797]]),
 array([[   0, 6204, 6311, 2131, 3276],
        [   1, 6204, 6311, 7161, 9490],
        [   2, 6311, 6204,  431, 5942],
        ...,
        [9997, 6311, 6204, 7583, 3065],
        [9998, 6311, 6204,   19, 2415],
        [9999, 6311, 6204, 9676, 7279]], dtype=int64))

In [19]:
#Creating my own review
review = ['Enjoyed this place. The food was amazing and I will always come back!']

#Transform the review
new = tfidf.transform(review)

In [20]:
print(new)

  (0, 3273)	0.3065319279844009
  (0, 1807)	0.3075349236742124
  (0, 1547)	0.6245747221446255
  (0, 969)	0.4524694051549714
  (0, 198)	0.46544419827035394


In [21]:
#Adding review using NN
nn.kneighbors(new.todense())

(array([[1.        , 1.        , 1.07520169, 1.09107504, 1.09967335]]),
 array([[6204, 6311,  123, 3127, 5030]], dtype=int64))

In [25]:
#pull up review match
data[5030]

"The food here is amazing!, why isn't this place booming?  I don't understand - the service is amazing and the food is super tasty"

## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a pipeline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier. Use that pipeline to estimate a model to predict `stars`. Use the Pipeline to predict a star rating for your fake review from Part 2. 
2. Tune the entire pipeline with a GridSearch

In [31]:
#Create a Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

stars= yelp['stars']
vect = TfidfVectorizer(stop_words='english')
sgd= SGDClassifier()

In [35]:
#Define Pipeline
pipe= Pipeline([
    ('vect',vect),
    ('clf',sgd)
])

#Grid Search and Params
param ={
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (100, 500,1000),
    'clf__max_iter':(20, 10, 100)
}

grid_search = GridSearchCV(pipe, param, cv=5,
                          n_jobs=-1,
                          verbose=1)
grid_search.fit(data,stars)

Fitting 5 folds for each of 54 candidates, totalling 270 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   48.7s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 270 out of 270 | elapsed:  4.5min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        no

In [37]:
#Check Score
grid_search.best_score_

0.5972

In [38]:
#Predict my review rating
grid_search.predict(review)

array([5], dtype=int64)

## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
    - Keep the `iterations` parameter at or below 5 to reduce run time
    - The `workers` parameter should match the number of physical cores on your machine.
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

In [39]:
from gensim.models import LdaMulticore
from gensim.corpora import Dictionary

Learn the vocabulary of the yelp data:

In [41]:
id2word = corpora.Dictionary(yelp['tokens'])

In [45]:
id2word.filter_extremes(no_below=5, no_above=0.95)

In [46]:
len(id2word.keys())

264

Create a bag of words representation of the entire corpus

In [48]:
corpus = [id2word.doc2bow(text) for text in yelp['tokens']]

Your LDA model should be ready for estimation: 

In [49]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   iterations=5,
                   workers=4,
                   num_topics = 10 # You can change this parameter
                  )

In [50]:
lda.print_topics()

[(0,
  '0.107*"be" + 0.105*"the" + 0.087*"and" + 0.053*"a" + 0.049*"to" + 0.042*"-PRON-" + 0.030*"not" + 0.027*"have" + 0.026*"of" + 0.021*"in"'),
 (1,
  '0.114*"the" + 0.098*"and" + 0.093*"be" + 0.051*"a" + 0.045*"to" + 0.037*"-PRON-" + 0.027*"not" + 0.027*"have" + 0.024*"of" + 0.023*"that"'),
 (2,
  '0.127*"be" + 0.080*"the" + 0.069*"and" + 0.062*"a" + 0.059*"to" + 0.042*"-PRON-" + 0.031*"not" + 0.028*"of" + 0.025*"in" + 0.024*"for"'),
 (3,
  '0.129*"be" + 0.074*"the" + 0.073*"and" + 0.056*"to" + 0.043*"-PRON-" + 0.041*"a" + 0.041*"of" + 0.030*"have" + 0.027*"in" + 0.023*"not"'),
 (4,
  '0.114*"be" + 0.091*"the" + 0.067*"and" + 0.063*"to" + 0.049*"-PRON-" + 0.041*"a" + 0.038*"of" + 0.035*"have" + 0.034*"not" + 0.021*"this"'),
 (5,
  '0.107*"be" + 0.107*"the" + 0.062*"and" + 0.049*"a" + 0.045*"to" + 0.040*"-PRON-" + 0.039*"have" + 0.031*"for" + 0.026*"of" + 0.025*"that"'),
 (6,
  '0.107*"the" + 0.084*"be" + 0.064*"and" + 0.043*"to" + 0.042*"a" + 0.042*"-PRON-" + 0.039*"of" + 0.030*"fo

In [51]:
words = [re.findall('"([^"]*)"',t[1]) for t in lda.print_topics()]

In [52]:
topics = [' '.join(t[0:5]) for t in words]

In [53]:
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
be the and a to

------ Topic 1 ------
the and be a to

------ Topic 2 ------
be the and a to

------ Topic 3 ------
be the and to -PRON-

------ Topic 4 ------
be the and to -PRON-

------ Topic 5 ------
be the and a to

------ Topic 6 ------
the be and to a

------ Topic 7 ------
the be and a to

------ Topic 8 ------
be the and a to

------ Topic 9 ------
be the a to and



Create 1-2 visualizations of the results

In [54]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

In [57]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.002172 -0.001570       1        1  10.097781
4     -0.003493  0.001096       2        1  10.077342
8      0.000283  0.001055       3        1  10.059372
2     -0.003193  0.002275       4        1  10.004599
0      0.000614  0.003290       5        1   9.985537
1      0.004988  0.003075       6        1   9.984950
5      0.002198 -0.001429       7        1   9.982057
9     -0.001003 -0.005831       8        1   9.968402
3     -0.005126  0.000109       9        1   9.935678
6      0.002560 -0.002071      10        1   9.904291, topic_info=   Category          Freq   Term         Total  loglift  logprob
19  Default  55126.000000    the  55126.000000  30.0000  30.0000
6   Default  38800.000000    and  38800.000000  29.0000  29.0000
7   Default  61184.000000     be  61184.000000  28.0000  28.0000
42  Default  16384.000000     of  16384.000000  27.0000  27.0000
2   Default  28084.000000      a  28084.000000  26.0000  26.0000
..      ...           ...    ...           ...      ...      ...
11  Topic10   1469.498291   have  17225.554688  -0.1493  -3.6051
30  Topic10    786.133362    but   8357.934570  -0.0516  -4.2307
32  Topic10    817.491882     do   8882.559570  -0.0734  -4.1915
25  Topic10    386.717743  would   3905.792480  -0.0003  -4.9401
47  Topic10    451.333282     so   5925.013672  -0.2625  -4.7856

[649 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
162       1  0.134517         '
162       2  0.134517         '
162       3  0.134517         '
162       4  0.134517         '
162       5  0.134517         '
...     ...       ...       ...
244       6  0.104157  yourself
244       7  0.104157  yourself
244       8  0.104157  yourself
244       9  0.104157  yourself
244      10  0.104157  yourself

[2189 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 5, 9, 3, 1, 2, 6, 10, 4, 7])

'''
This topic model is visualizing the relationships between each topic. All of the topics that overlaps each other are inner like tokens that appear in the same domain. The bigger the distribution, you can see it being represented by the size of each circle. You can see that there a lot of relevant topics that are similar in each review. 
'''

## Stretch Goals

Complete one of more of these to push your score towards a three: 
* Incorporate named entity recognition into your analysis
* Compare vectorization methods in the classification section
* Analyze more (or all) of the yelp dataset - this one is v. hard. 
* Use a generator object on the reviews file - this would help you with the analyzing the whole dataset.
* Incorporate any of the other yelp dataset entities in your analysis (business, users, etc.)